# Assignment 2 Part 1 Questions #
1. A data warehouse for a music streaming company consists of the dimensions song, user, time (time and date of when the user listened to a song), and the two measures count (how many times a user listened to the song) and fee (fee paid by the streaming company to the artist every time a user listens to that song).

1. Draw a schema diagram for the above data warehouse using a star schema.

1. Starting with the base cuboid [time, user, song], what specific OLAP operations should be performed in order to list the total fee collected for a given song for a given month of a given year (e.g. October 2021)?

1. Assume that the time dimension has 4 levels: day, month, quarter, year; and that the song and user dimensions both have 1 level (not including the virtual level 'all'). How many cuboids will this cube contain (including the base and apex cuboids)?

1. Suppose that a car rental company has a data warehouse that holds record ID lists of vehicles in terms of brands (Audi, Ford, Mini) and store branches (Tower Hamlets, Newham, Hackney). Each record consists of a combination of vehicle brand and branch, and records for all combinations exist. We would like to index the OLAP data using bitmap indices. Write down the base table for record IDs, and the corresponding bitmap index table for vehicle brand.

1. Using the same CSV file and data cube in the above lab tutorial, modify the "tutorial_model.json" file to include aggregate measures for the minimum and maximum amount in the data cube. Using these implemented aggregate measures, produce the values for the minimum and maximum amount in the data per year. Make sure to show your workings in the PDF report. You can read the Cubes package documentation for assistance in this task.

1. Using the CSV file "country-income.csv" (found in the week 5 supplementary lab documents), perform the following:

1. Load the CSV file using Cubes, create a JSON file for the data cube model, and create a data cube for the data. Use as dimensions the region, age, and online shopper fields. Use as measure the income. Define aggregate functions in the data cube model for the total, average, minimum, and maximum income. In your PDF report, show the relevant scripts and files created.

1. Using the created data cube and data cube model, produce aggregate results for: the whole data cube; results per region; results per online shopping activity; and results for all people aged between 40 and 50.

# Assignment 2
# Part 1 of 2
## Question 1
### A
![Star_Schema](Star_Schema.png)

### B
Roll up user to All. Song left as normal, time roll up from day to month and slice the desired month (October 2021).

### C
$\prod_{i=1}^{n} (L_i + 1)$

$(4+1)(1+1)(1+1)$

$5\times2\times2 = 20 cuboids$

## Question 2

### Base

| RID | Brand | Branch |
|:----|:------|:------:|
| 01  | Audi  | Tower Hamlets |
| 02  | Audi  | Newham |
| 03  | Audi  | Hackney |
| 04  | Ford  | Tower Hamlets |
| 05  | Ford  | Newham |
| 06  | Ford  | Hackney |
| 07  | Mini  | Tower Hamlets |
| 08  | Mini  | Newham |
| 09  | Mini  | Hackney |

### Bitmap for brands

| RID | Audi | Ford | Mini |
|:---:|:----:|:----:|:----:|
| 01  | 1 | 0 | 0 |
| 02  | 1 | 0 | 0 |
| 03  | 1 | 0 | 0 |
| 04  | 0 | 1 | 0 |
| 05  | 0 | 1 | 0 |
| 06  | 0 | 1 | 0 |
| 07  | 0 | 0 | 1 |
| 08  | 0 | 0 | 1 |
| 09  | 0 | 0 | 1 |

## Question 3
### json code
```
{
    "dimensions": [
        {
         "name":"item",
         "levels": [
                {
                    "name":"category",
                    "label":"Category",
                    "attributes": ["category", "category_label"]
                },
                {
                    "name":"subcategory",
                    "label":"Sub-category",
                    "attributes": ["subcategory", "subcategory_label"]
                },
                {
                    "name":"line_item",
                    "label":"Line Item",
                    "attributes": ["line_item"]
                }
            ]
        },
        {"name":"year", "role": "time"}
    ],
    "cubes": [
        {
            "name": "ibrd_balance",
            "dimensions": ["item", "year"],
            "measures": [{"name":"amount", "label":"Amount"}],
            "aggregates": [
                    {
                        "name": "amount_sum",
                        "function": "sum",
                        "measure": "amount"
                    },
                    {
                        "name": "record_count",
                        "function": "count"
                    },
                    {
                        "name": "amount_min",
                        "function": "min",
                        "measure": "amount"
                    },
                    {
                        "name": "amount_max",
                        "function": "max",
                        "measure": "amount"
                    }
                ],
            "mappings": {
                          "item.line_item": "line_item",
                          "item.subcategory": "subcategory",
                          "item.subcategory_label": "subcategory_label",
                          "item.category": "category",
                          "item.category_label": "category_label"
                         },
            "info": {
                "min_date": "2010-01-01",
                "max_date": "2010-12-31"
            }
        }
    ]
}
```

In [9]:
from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv
from cubes import Workspace
import cubes as cubes

In [2]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                     )

In [3]:
workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")

In [4]:
workspace.import_model("tutorial_model.json")
cube = workspace.cube("ibrd_balance")
browser = workspace.browser(cube)

In [55]:
result = browser.aggregate(drilldown=["year"])
for record in result:
    print(str(record["year"]) + " : Minimum = " + str(record["amount_min"]) + " : Maximum = " +  str(record["amount_max"]))

2009 : Minimum = -1683 : Maximum = 110040
2010 : Minimum = -3043 : Maximum = 128577


## Question 4
### A
#### JSON Code
`{
    "dimensions": [
        {"name":"region"},
		{"name":"age"},
		{"name":"online_shop"}		
    ],
    "cubes": [
        {
            "name": "country-income",
            "dimensions": ["region", "age", "online_shop"],
            "measures": [{"name":"income", "label":"income"}],
            "aggregates": [
                    {
                        "name": "total",
                        "function": "sum",
                        "measure": "income"
                    },
                    {
                        "name": "average",
                        "function": "avg",
                        "measure": "income"
                    },
                    {
                        "name": "min",
                        "function": "min",
                        "measure": "income"
                    },
                    {
                        "name": "max",
                        "function": "max",
                        "measure": "income"
                    }
                ]
        }
    ]
}
`

### B
#### Set up

In [56]:
engine = create_engine('sqlite:///data2.sqlite')
create_table_from_csv(engine,
                      "country-income.csv",
                      table_name="country-income",
                      fields=[
                          ("region", "string"),
                          ("age", "integer"),
                          ("income", "integer"),
                          ("online_shop", "string")],
                      create_id=True
                     )

In [57]:
workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data2.sqlite")

In [58]:
workspace.import_model("country-income.json")

In [59]:
cube = workspace.cube("country-income")

In [60]:
browser = workspace.browser(cube)

#### Whole Data Cube

In [62]:
agg = browser.aggregate()
agg.summary

{'total': 768200, 'average': 76820.0, 'min': 57600, 'max': 99600}

#### Per Region

In [10]:
region = browser.aggregate(drilldown=["region"])
for record in region:
    print(record)

{'region': 'Brazil', 'total': 193200, 'average': 64400.0, 'min': 57600, 'max': 73200}
{'region': 'India', 'total': 331200, 'average': 82800.0, 'min': 69600, 'max': 94800}
{'region': 'USA', 'total': 243800, 'average': 81266.66666666667, 'min': 64800, 'max': 99600}


#### Online Shopper

In [11]:
online = browser.aggregate(drilldown=["online_shop"])
for record in online:
    print(record)

{'online_shop': 'No', 'total': 386400, 'average': 77280.0, 'min': 62400, 'max': 99600}
{'online_shop': 'Yes', 'total': 381800, 'average': 76360.0, 'min': 57600, 'max': 94800}


#### Aged 40-50

In [13]:
import cubes as cubes

cuts = [cubes.RangeCut("age", [40], [50])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["age"])
print(result.summary)

{'total': 451400, 'average': 75233.33333333333, 'min': 62400, 'max': 86400}


# Part 2 of 2
# Questions
1. Consider a dataset 
 that contains only two observations 
 and 
. Suppose that the class of the first observation is 
 and that the class of the second observation is 
. How would a 1-nearest neighbour classifier based on the Euclidean distance classify the observation 
? What are the distances between this new observation and each observation in the dataset?
1. Consider a dataset 
 that only contains observations of two different classes. Explain why a 
-nearest neighbour classifier does not need a tie-breaking policy when 
 is odd.
1. Explain why a classifier that obtains an accuracy of 
 can be terrible for some datasets.
1. Consider a classifier tasked with predicting whether an observation belongs to class 
 (positive class). Suppose that this classifier has precision 
 and recall 
 on a test dataset. If this classifier predicts that an observation does not belong to class 
, should it be trusted? Should it be trusted if it predicts that the observation belongs to class 
?
1. Based on the confusion matrix shown in this lab notebook, what is the pair of classes that is most confusing for the 
-nearest neighbour classifier trained in the previous sections?
1. Train a support vector machine classifier using the same (subsampled) training dataset used in the previous sections and compute its accuracy on the corresponding test dataset. You can use the default hyperparameters for the class SVC from sklearn.svm. Show the code in the report.
1. Using the same (subsampled) training dataset used in the previous sections, employ GridSearchCV to find the best hyperparameter settings based on 5-fold cross-validation for a RandomForestClassifier. Consider n_estimators 
 and max_features 
. Use the default values for the remaining hyperparameters. Compute the accuracy of the best model on the corresponding test dataset. Show the code in the report.


## Question 1
$\mathbf{x} = (-2, 3)$ will be classified as class 1, because it is closer to (-1,1) than it is (1, -1)

For (-1, 1) (Class 1): $\sqrt{(-2 - -1)^2 + (3 - 1)^2} = \sqrt{5} ≈ 2.2$

For (1, -1) (Class 0): $\sqrt{(-2 - 1)^2 + (3 - -1)^2} = \sqrt{25} = 5$ 

## Question 2
The hyper parameter $k$ means that the algorithm looks at k-number of nearest neighbours. A new point is assigned to the class of the majority of the nearest neighbours. When there are only two classes, if $k$ is three, five or any odd number, there will never be a tie break because there will never be an equal amount of each class. For example, 3/2 = 1.5; there is only ever an integer amount in each class. However, if k was 4, then there could be a tie break because you may have two neighbours of each class. Thus, tie breaks will not happen if $k$ is not a multiple of the number of classes.

## Question 3
Every attribute is given an equal weight, therefore the classifier of 99.9% may be vulnerable to noise. In a terrible dataset where there is an imbalance of classes, like 99.9% of one class and 0.1% of the other, then the accuracy will always be 99.9% because of this imbalance. Although the accuracy is high, it is pointless.

## Question 4
A precision of 1.0 shows that there are no false positives, and so all positives are truly positive. This means you can fully trust in the classifier if it predicts a belonging to class y.

However, a recall of 0.1 suggests that only 10% of the truly positive observations are classified as positive. So although the precision suggests everything it detects as positive is positive, from the recall value it is missing out 90% of all the positives, so you should not trust it to tell you that something *is not* positive.

## Question 5
Class 9 has the second lowest precision and first lowest recall, and from the confusion matrix, the classifier is classifying 4 true class 9 observations as class 4. Similarly, 3 true class 4 observations are being classified as class 9. Therefore, class 4 and 9 are the most confusing.

## Question 6

In [1]:
import gzip
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

f = gzip.open('mnist.pkl.gz', 'rb')
X, y = pickle.load(f, encoding='latin1')[0]
f.close()

sample_size = 2000
X, y = X[:sample_size], y[:sample_size]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [4]:
from sklearn.svm import SVC
clf = SVC(random_state=1)
clf.fit(X_train, y_train)
print('Train dataset accuracy (Support Vector Machines): {0}.'.format(clf.score(X_train, y_train)))
print('Test dataset accuracy (Support Vector Machines): {0}.'.format(clf.score(X_test, y_test)))

Train dataset accuracy (Support Vector Machines): 0.979375.
Test dataset accuracy (Support Vector Machines): 0.9275.


## Question 7

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

parameters = {'n_estimators': [50, 100, 200], 'max_features': [0.1, 0.25]}

rfc = RandomForestClassifier()
rfc_gs = GridSearchCV(rfc, parameters, cv=5)
rfc_gs.fit(X_train, y_train)

print('Best hyperparameter setting: {0}.'.format(rfc_gs.best_estimator_))
print('Average accuracy across folds of best hyperparameter setting: {0}.'.format(rfc_gs.best_score_))
print('Test dataset accuracy of best hyperparameter setting: {0}.'.format(rfc_gs.score(X_test, y_test)))

Best hyperparameter setting: RandomForestClassifier(max_features=0.1, n_estimators=200).
Average accuracy across folds of best hyperparameter setting: 0.911875.
Test dataset accuracy of best hyperparameter setting: 0.91.
